## Set up

In [ ]:
pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 17.4 MB/s eta 0:00:00


In [ ]:
# FLOODNET SEGMENTATION

import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import SegformerForSemanticSegmentation
from transformers import SegformerImageProcessor  # ĐÃ THAY SegformerFeatureExtractor → ImageProcessor
from tqdm import tqdm
import numpy as np
from torchmetrics import JaccardIndex # pip install torchmetrics nếu chưa có
import glob

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# Tạo folder checkpoints trong Drive (nếu chưa có)
!mkdir -p "/content/drive/MyDrive/CODE/KHKT/checkpoints"

# Tạo symbolic link từ /content/checkpoints → Drive
!ln -sfn "/content/drive/MyDrive/CODE/KHKT/checkpoints" "/content/checkpoints"

# Kiểm tra (phải thấy đường dẫn trỏ về Drive)
!ls -la /content/checkpoints

Mounted at /content/drive
lrwxrwxrwx 1 root root 44 Nov 27 01:20 /content/checkpoints -> /content/drive/MyDrive/CODE/KHKT/checkpoints


## Cấu hình Class

In [ ]:
id2label = {
    0: "background",
    1: "building_flooded",
    2: "building_non_flooded",
    3: "road_flooded",
    4: "road_non_flooded",
    5: "water",
    6: "tree",
    7: "vehicle",
    8: "pool",
    9: "grass",
}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

def rgb_mask_to_label(mask_rgb: np.ndarray) -> np.ndarray:
    """
    FloodNet dùng mask dạng grayscale:
    - 0,1,3,5,6,9,... chính là class_id
    Mask thường load ra (H,W,3) nhưng cả 3 kênh đều = nhau (x,x,x),
    nên chỉ cần lấy 1 kênh là đủ.
    """
    if mask_rgb.ndim == 3:
        gray = mask_rgb[:, :, 0]
    else:
        gray = mask_rgb  # đã là (H,W)

    return gray.astype(np.int64)

## Tiền xử lý dataset

In [ ]:
class FloodNetDataset(Dataset):
    def __init__(self, root_dir, split="train", processor=None, val_split=0.2):
        self.root_dir = root_dir
        self.split = split.lower()
        self.processor = processor
        self.has_mask = True  # Luôn có mask vì dùng từ labeled

        # Lấy tất cả ảnh labeled từ train
        img_dir_f = os.path.join(root_dir, "train", "labeled", "flooded", "img")
        img_dir_nf = os.path.join(root_dir, "train", "labeled", "non-flooded", "img")
        all_images = []
        for d in [img_dir_f, img_dir_nf]:
            if os.path.exists(d):
                all_images += [f for f in os.listdir(d) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        # Chia ngẫu nhiên: train/val (seed để reproducible)
        import random
        random.seed(42)  # Fix seed để kết quả ổn định
        random.shuffle(all_images)
        split_idx = int(len(all_images) * (1 - val_split))

        if self.split == "train":
            self.image_files = all_images[:split_idx]  # 80%
        else:  # validation
            self.image_files = all_images[split_idx:]  # 20%

        print(f"[{split.upper()}] Found {len(self.image_files)} images (from labeled split)")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]

        # === TRAIN ===
        if self.split == "train":
            # flooded img?
            flooded_path = os.path.join(self.root_dir, "train/labeled/flooded/img", img_name)
            if os.path.exists(flooded_path):
                img_path = flooded_path
                mask_dir = os.path.join(self.root_dir, "train/labeled/flooded/mask")
            else:
                # non-flooded img?
                img_path = os.path.join(self.root_dir, "train/labeled/non-flooded/img", img_name)
                mask_dir = os.path.join(self.root_dir, "train/labeled/non-flooded/mask")

            if not os.path.exists(img_path):
                raise FileNotFoundError(f"Train image not found: {img_path}")

            # mask thật
            mask_path = os.path.join(mask_dir, f"{os.path.splitext(img_name)[0]}_lab.png")
            if not os.path.exists(mask_path):
                raise FileNotFoundError(f"Train mask not found: {mask_path}")

            image = Image.open(img_path).convert("RGB")
            mask_rgb = np.array(Image.open(mask_path).convert("RGB"))
            mask_label = rgb_mask_to_label(mask_rgb)

            # Encode train
            encoded = self.processor(
                images=image,
                segmentation_maps=mask_label,
                return_tensors="pt"
            )
            return {k: v.squeeze(0) for k, v in encoded.items()}

        # === VALIDATION ===
        else:
            img_path = os.path.join(self.root_dir, "validation/img", img_name)

            if not os.path.exists(img_path):
                raise FileNotFoundError(f"Validation image not found: {img_path}")

            # pseudo mask
            pseudo_path = os.path.join(
                self.root_dir,
                "validation/pseudo_mask",
                f"{os.path.splitext(img_name)[0]}_lab.png"
            )

            if not os.path.exists(pseudo_path):
                # Nếu không có pseudo mask -> bỏ qua ảnh này
                return None

            image = Image.open(img_path).convert("RGB")
            mask_rgb = np.array(Image.open(pseudo_path).convert("RGB"))
            mask_label = rgb_mask_to_label(mask_rgb)

            encoded = self.processor(
                images=image,
                segmentation_maps=mask_label,
                return_tensors="pt"
            )
            return {k: v.squeeze(0) for k, v in encoded.items()}

## Train model

In [ ]:
def train():
    root_dir = "/content/drive/MyDrive/CODE/KHKT/FloodNet/"

    processor = SegformerImageProcessor.from_pretrained(
        "nvidia/segformer-b0-finetuned-ade-512-512",
        do_reduce_labels=False,
        size={"height": 512, "width": 512}
    )

    # === TRAIN DATASET (bắt buộc) ===
    train_dataset = FloodNetDataset(root_dir, split="train", processor=processor)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True,
                              num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)

    # === VALIDATION DATASET (tùy chọn - chỉ tạo nếu có ảnh) ===
    val_images = glob.glob(f"{root_dir}/validation/img/*.jpg") + glob.glob(f"{root_dir}/validation/img/*.png")
    if len(val_images) == 0:
        print("Không tìm thấy ảnh validation -> chỉ train, không validate")
        val_loader = None
        val_size = 0
    else:
        print(f"Tìm thấy {len(val_images)} ảnh validation -> tạo val_dataset")
        val_dataset = FloodNetDataset(root_dir, split="validation", processor=processor)
        val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False,
                                num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
        val_size = len(val_dataset)

    print(f"Train: {len(train_dataset)} ảnh | Validation: {val_size} ảnh")

    # === MODEL ===
    model = SegformerForSemanticSegmentation.from_pretrained(
        "nvidia/segformer-b0-finetuned-ade-512-512",
        num_labels = 10,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True,
    ).cuda()

    optimizer = torch.optim.AdamW(model.parameters(), lr=8e-5, weight_decay=0.01)
    total_steps = len(train_loader) * 15
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=2e-4, total_steps=total_steps, pct_start=0.1)

    os.makedirs("checkpoints", exist_ok=True)

    # === TRAINING LOOP ===
    for epoch in range(1, 16):  # 15 epochs
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch}/15 [TRAIN]"):
            outputs = model(pixel_values=batch["pixel_values"].cuda(),
                          labels=batch["labels"].cuda())
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        print(f"EPOCH {epoch}/15 | Train Loss: {avg_train_loss:.4f}\n")

        # === LƯU MODEL ===
        if epoch % 3 == 0 or epoch == 15:
            save_path = f"checkpoints/segformer_floodnet_epoch{epoch}"
            model.save_pretrained(save_path)
            processor.save_pretrained(save_path)
            print(f"Đã lưu: {save_path}")

        # Luôn lưu latest
        model.save_pretrained("checkpoints/latest_segformer")
        processor.save_pretrained("checkpoints/latest_segformer")

    print("\nHOÀN TẤT! Model tốt nhất thường là epoch 12–15")
    print("→ Dùng: checkpoints/segformer_floodnet_epoch12 hoặc epoch15 để inference")

## Main

In [ ]:
if __name__ == "__main__":
    # Tạo folder lưu checkpoint
    os.makedirs("checkpoints", exist_ok=True)
    train()

[TRAIN] Found 318 images (from labeled split)
Tìm thấy 454 ảnh validation → tạo val_dataset
[VALIDATION] Found 80 images (from labeled split)
Train: 318 ảnh | Validation: 80 ảnh


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([10]) in the model instantiated
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([10, 256, 1, 1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/15 [TRAIN]: 100%|██████████| 40/40 [00:58<00:00,  1.47s/it]



EPOCH 1/15 | Train Loss: 2.0525


Epoch 2/15 [TRAIN]: 100%|██████████| 40/40 [00:58<00:00,  1.45s/it]



EPOCH 2/15 | Train Loss: 1.2351


...


EPOCH 13/15 | Train Loss: 0.2979


Epoch 14/15 [TRAIN]: 100%|██████████| 40/40 [00:57<00:00,  1.45s/it]



EPOCH 14/15 | Train Loss: 0.2952


Epoch 15/15 [TRAIN]: 100%|██████████| 40/40 [00:58<00:00,  1.45s/it]



EPOCH 15/15 | Train Loss: 0.2998
Đã lưu: checkpoints/segformer_floodnet_epoch15

HOÀN TẤT! Model tốt nhất thường là epoch 12–15
→ Dùng: checkpoints/segformer_floodnet_epoch12 hoặc epoch15 để inference
